# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет.

## Описание данных

Данные находятся в файле `/datasets/Churn.csv`

### Признаки
- `RowNumber` — индекс строки в данных
- `CustomerId` — уникальный идентификатор клиента
- `Surname` — фамилия
- `CreditScore` — кредитный рейтинг
- `Geography` — страна проживания
- `Gender` — пол
- `Age` — возраст
- `Tenure` — количество недвижимости у клиента
- `Balance` — баланс на счёте
- `NumOfProducts` — количество продуктов банка, используемых клиентом
- `HasCrCard` — наличие кредитной карты
- `IsActiveMember` — активность клиента
- `EstimatedSalary` — предполагаемая зарплата

### Целевой признак
- `Exited` — факт ухода клиента

 # Содержание

### [1 Подготовка данных](#1-bullet)
### [2 Исследование задачи](#2-bullet)
### [3 Исследуем модели](#3-bullet)

- #### [3.1 LogisticRegression (Логистическая регрессия)](#4-bullet)
- #### [3.2 DecisionTreeClassifier (Решающее дерево)](#5-bullet)
- #### [3.3 RandomForestClassifier (Случайный лес)](#6-bullet)

### [4 Борьба с дисбалансом](#7-bullet)

- #### [4.1 С помощью взвешивания классов](#8-bullet)
- #### [4.2 Увеличение выборки (upsampling)](#9-bullet)
- #### [4.3 Уменьшение выборки (downsampling)](#10-bullet)

### [5. Тестирование итоговой модели и результаты](#11-bullet)

## 1 Подготовка данных <a class="anchor" id="1-bullet"></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, roc_curve
from sklearn.utils import shuffle

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/datasets/Churn.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [6]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [7]:
df.columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited']

Предположительно, столбец `row_number` лишний.

In [8]:
df['row_number'].duplicated().sum()

0

In [9]:
df = df.drop(columns = ['row_number'])

In [10]:
for i in ['surname', 'geography', 'gender']:
    df[i] = df[i].str.lower()

In [11]:
df.head()

,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,15634602,hargrave,619,france,female,42,2.0,0.00,1,1,1,101348.88,1
1,15647311,hill,608,spain,female,41,1.0,83807.86,1,0,1,112542.58,0
2,15619304,onio,502,france,female,42,8.0,159660.80,3,1,0,113931.57,1
3,15701354,boni,699,france,female,39,1.0,0.00,2,0,0,93826.63,0
4,15737888,mitchell,850,spain,female,43,2.0,125510.82,1,1,1,79084.10,0


В данных присутствуют пропуски в столбце `tenure`.

In [12]:
df['tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

In [13]:
df.corr()

,customer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
customer_id,1.000000,0.005308,0.009497,-0.021418,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
credit_score,0.005308,1.000000,-0.003965,-0.000062,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
age,0.009497,-0.003965,1.000000,-0.013134,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
tenure,-0.021418,-0.000062,-0.013134,1.000000,-0.007911,0.011979,0.027232,-0.032178,0.010520,-0.016761
balance,-0.012419,0.006268,0.028308,-0.007911,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
num_of_products,0.016972,0.012238,-0.030680,0.011979,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
has_cr_card,-0.014025,-0.005458,-0.011721,0.027232,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
is_active_member,0.001665,0.025651,0.085472,-0.032178,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
estimated_salary,0.015271,-0.001384,-0.007201,0.010520,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097
exited,-0.006248,-0.027094,0.285323,-0.016761,0.118533,-0.047820,-0.007138,-0.156128,0.012097,1.000000


Сильной корреляции `tenure` с другими признаками не выявлено, удалим строки со значением `tenure` = 0.

In [14]:
df = df.loc[df['tenure'].notnull()]

In [15]:
#df.loc[df['tenure'].isna(), 'tenure'] = df['tenure'].median()

Изменим тип данных в столбцах `tenure`, `balance` и `estimated_salary` на целочисленный.

In [16]:
for column in ['tenure', 'balance', 'estimated_salary']:
    df[column] = df[column].astype(int)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 13 columns):
customer_id         9091 non-null int64
surname             9091 non-null object
credit_score        9091 non-null int64
geography           9091 non-null object
gender              9091 non-null object
age                 9091 non-null int64
tenure              9091 non-null int64
balance             9091 non-null int64
num_of_products     9091 non-null int64
has_cr_card         9091 non-null int64
is_active_member    9091 non-null int64
estimated_salary    9091 non-null int64
exited              9091 non-null int64
dtypes: int64(10), object(3)
memory usage: 994.3+ KB


In [18]:
df.head(2)

,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,15634602,hargrave,619,france,female,42,2,0,1,1,1,101348,1
1,15647311,hill,608,spain,female,41,1,83807,1,0,1,112542,0


## 2 Исследование задачи <a class="anchor" id="2-bullet"></a>

Для создания модели и ее обучения столбцы `customer_id` и `surname` не нужны. Удалим и их.

In [19]:
df = df.drop(columns = ['customer_id', 'surname'])
df.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,france,female,42,2,0,1,1,1,101348,1
1,608,spain,female,41,1,83807,1,0,1,112542,0
2,502,france,female,42,8,159660,3,1,0,113931,1
3,699,france,female,39,1,0,2,0,0,93826,0
4,850,spain,female,43,2,125510,1,1,1,79084,0


Осталось 2 категориальных признака: `geography` и `gender`. Закодируем их с помощью `get_dummies`.

In [20]:
df = pd.get_dummies(data = df, columns = ['geography', 'gender'], drop_first=True)
df.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_germany,geography_spain,gender_male
0,619,42,2,0,1,1,1,101348,1,0,0,0
1,608,41,1,83807,1,0,1,112542,0,0,1,0
2,502,42,8,159660,3,1,0,113931,1,0,0,0
3,699,39,1,0,2,0,0,93826,0,0,0,0
4,850,43,2,125510,1,1,1,79084,0,0,1,0


Разделим данные на выборки в соотношении :

- `train` - обучающую (60% данных),
- `valid` - валидационную (20% данных),
- `test` - тестовую (20% данных).

In [21]:
target = df['exited']
features = df.drop('exited', axis=1)

In [22]:
# разделим данные features и target на две части (train и test) в соотношении 60% к 40% 
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

In [23]:
# разделим данные features_test и target_test на две части (test и valid) в соотношении 50% к 50% 
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test, test_size=0.5, random_state=12345)

In [24]:
target_train.value_counts()

0    4328
1    1126
Name: exited, dtype: int64

In [25]:
target_valid.value_counts()

0    1468
1     351
Name: exited, dtype: int64

Данные несбалансированы. Дисбаланс имеет соотношение где-то 4:1.<br>
Обучим сначала модель без учета дисбаланса.

## 3 Исследуем модели <a class="anchor" id="3-bullet"></a>

Рассмотрим три модели:
- `DecisionTreeClassifier` (Решающее дерево),
- `RandomForestClassifier` (Случайный лес),
- `LogisticRegression` (Логистическая регрессия).

### 3.1 `LogisticRegression` (Логистическая регрессия) <a class="anchor" id="4-bullet"></a>

In [ ]:
%%time

best_result = 0
for solver in ['newton-cg','lbfgs','sag','saga','liblinear']:
    model = LogisticRegression(random_state=12345, solver = solver, max_iter=4000)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid)
    if best_result <= f1:
        best_result = f1
        best_solver = solver

print('Лучший результат F1 =', best_result, 'при solver = ',best_solver)

### 3.2 `DecisionTreeClassifier` (Решающее дерево) <a class="anchor" id="5-bullet"></a>

In [ ]:
%%time

best_result = 0
for depth in range(1, 10): 
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid)
    if best_result <= f1:
        best_result = f1
        best_depth = depth
        
print('Лучший результат F1 =', best_result, 'при depth = ', best_depth)

### 3.3 `RandomForestClassifier` (Случайный лес) <a class="anchor" id="6-bullet"></a>

In [ ]:
%%time

# При использовании Jupyterhub на https://praktikum.yandex.ru/ 
# CPU times: user 1min 20s, sys: 136 ms, total: 1min 20s
# Wall time: 1min 20s

best_result = 0
for depth in range(90,110,5): 
    for estimators in range(100,401,100):
        model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions_valid)
        if best_result <= f1:
            best_result = f1
            best_depth = depth
            best_n_estimators = estimators
            
print('Лучший результат F1 =', best_result, 'при depth = ', best_depth, 'и при n_estimators = ', estimators)

Для дальнейшей работы выбираем модель `RandomForestClassifier`, т.к. она показывает наилучший результат. Стоит отметить, что модель `DecisionTreeClassifier` также показала достаточно высокий результат и вычислялась намного быстрее, чем `RandomForestClassifier`. `LogisticRegression (Логистическая регрессия)` показала худший результат.

## 4 Борьба с дисбалансом <a class="anchor" id="7-bullet"></a>

Рассмотрим 3 способа для борьбы с дисбалансом:
- с помощью взвешивания классов (<b>class_weight = 'balanced'</b> для модели `RandomForestClassifier`),
- с помощью увеличения выборки (<b>upsampling</b>),
- с помощью уменьшения выборки (<b>downsampling </b>).

### 4.1 С помощью взвешивания классов (`class_weight = 'balanced'` для модели `RandomForestClassifier`) <a class="anchor" id="8-bullet"></a>

In [ ]:
model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345, class_weight = 'balanced')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC =', auc_roc)
print('Значение F1 =', f1)

### 4.2 Увеличение выборки (<b>upsampling</b>) <a class="anchor" id="9-bullet"></a>

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [ ]:
target_upsampled.value_counts()

In [ ]:
model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345)
model.fit(features_upsampled, target_upsampled)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC =', auc_roc)
print('Значение F1 =', f1)

### 4.3 Уменьшение выборки (<b>downsampling</b>) <a class="anchor" id="10-bullet"></a>

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.3)

In [ ]:
target_downsampled.value_counts()  

In [ ]:
model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345)
model.fit(features_downsampled, target_downsampled)
predictions_valid = model.predict(features_valid)

f1 = f1_score(target_valid, predictions_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('Значение AUC-ROC =', auc_roc)
print('Значение F1 =', f1)

Лучший результат получили при использовании техники `upsampling`. Для борьбы с дисбалансом остановимся на ней.

# 5. Тестирование итоговой модели <a class="anchor" id="11-bullet"></a>

In [ ]:
model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345)
model.fit(features_upsampled, target_upsampled)
predictions_test = model.predict(features_test)
f1 = f1_score(target_test, predictions_test)

probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_valid)

print('Значение AUC-ROC =', auc_roc)
print('Значение F1 =', f1)

In [ ]:
fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_valid) 
plt.figure()
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show() 

### Вывод:

- Для итоговой модели `RandomForestClassifier` (Случайный лес) получили значение метрики `F1 = 0.6172`.
- ROC - кривая находится выше диагонали. Значение `AUC-ROC` для итоговой модели `0.8631`.